# Evaluate Data Quality Issues in the Data Provided
This notebook illustrates my process for assessing data quality when given new datasets. I included notes in Markdown along the way, to serve as a guide for my data cleaning and analysis.

In [1]:
import json
import pandas as pd
from collections import Counter

import sqlite3

# Receipt Table

In [2]:
receipts = pd.read_json('receipts.json', lines=True)
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [3]:
# Make a copy to use for creating Item table
receipts_for_items = receipts.copy()

The first step is to confirm that the primary key of the table is indeed unique.

In [4]:
print(len(receipts))

list_of_recids = [row['$oid'] for row in receipts['_id']]
print(len(set(list_of_recids)))

1119
1119


Though there are 1,119 rows of data in this table, many are missing values in various columns. Some missing values are likely due to receipts that have not finished processing, but others appear to be incorrect data. For example, when we only look at receipts that have been tagged as "Finished", there are still two receipts with no items on them.

In [5]:
receipts.count()

_id                        1119
bonusPointsEarned           544
bonusPointsEarnedReason     544
createDate                 1119
dateScanned                1119
finishedDate                568
modifyDate                 1119
pointsAwardedDate           537
pointsEarned                609
purchaseDate                671
purchasedItemCount          635
rewardsReceiptItemList      679
rewardsReceiptStatus       1119
totalSpent                  684
userId                     1119
dtype: int64

In [6]:
receipts[receipts['rewardsReceiptStatus']=='FINISHED'].count()

_id                        518
bonusPointsEarned          456
bonusPointsEarnedReason    456
createDate                 518
dateScanned                518
finishedDate               518
modifyDate                 518
pointsAwardedDate          514
pointsEarned               518
purchaseDate               518
purchasedItemCount         518
rewardsReceiptItemList     516
rewardsReceiptStatus       518
totalSpent                 518
userId                     518
dtype: int64

Looking deeper into only the finished receipts, several have issues with the items claimed to be included on them. Some receipts have a purchasedItemCount of zero. Others fail to match the quantityPurchased with the count of items in the rewardsReceiptItemList. Still others don't have a quantityPurchased label in their item lists at all.

In [7]:
finished_receipts = receipts[receipts['rewardsReceiptStatus']=='FINISHED']

In [8]:
finished_receipts[finished_receipts['purchasedItemCount']<1]

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
396,{'$oid': '6009eb000a7214ada2000003'},250.0,"Receipt number 3 completed, bonus point schedu...",{'$date': 1611262720000},{'$date': 1611262720000},{'$date': 1611262746000},{'$date': 1611262755000},{'$date': 1611262746000},250.0,{'$date': 1611187200000},0.0,NaN,FINISHED,0.0,6009e60450b3311194385009
424,{'$oid': '600aff160a720f053500000c'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1611333398000},{'$date': 1611333398000},{'$date': 1611333421000},{'$date': 1611333433000},{'$date': 1611333421000},500.0,{'$date': 1611273600000},0.0,NaN,FINISHED,0.0,600afb2a7d983a124e9aded0


In [9]:
finished_receipts = finished_receipts[finished_receipts['purchasedItemCount']>=1]

In [10]:
# A "mismatch" occurs when the quantity on the receipt is different from the sum of items on the receipt.
# A Key Error occurs when there is no quantity listed on the receipt.
for index,row in finished_receipts.iterrows():
    try:
        qp = sum([elem['quantityPurchased'] for elem in row['rewardsReceiptItemList']])
        quantity_match = (qp == row['purchasedItemCount'])
        if quantity_match == False:
            print('Mismatch at index: ' + str(index))
    except KeyError as e:
        print('KeyError at index: ' + str(index))

Mismatch at index: 4
KeyError at index: 19
KeyError at index: 24
KeyError at index: 26
KeyError at index: 72
KeyError at index: 87
KeyError at index: 133
Mismatch at index: 138
KeyError at index: 140
KeyError at index: 143
KeyError at index: 146
KeyError at index: 157
KeyError at index: 162
Mismatch at index: 163
KeyError at index: 178
KeyError at index: 191
Mismatch at index: 200
KeyError at index: 204
KeyError at index: 208
KeyError at index: 261
Mismatch at index: 378
KeyError at index: 379
Mismatch at index: 489
Mismatch at index: 534
KeyError at index: 551
KeyError at index: 564
KeyError at index: 569
KeyError at index: 651
KeyError at index: 736
KeyError at index: 895
KeyError at index: 948
Mismatch at index: 964


Another way to assess data quality is by checking if the dates are in a logical order.

In [11]:
for index,row in finished_receipts[~pd.isna(finished_receipts['finishedDate'])].iterrows():
    c_date = pd.to_datetime(row['createDate']['$date'], unit='ms')
    f_date = pd.to_datetime(row['finishedDate']['$date'], unit='ms')
    if c_date > f_date:
        print('Logical error at index: ' + str(index))

In [12]:
for index,row in finished_receipts[~pd.isna(finished_receipts['finishedDate'])].iterrows():
    p_date = pd.to_datetime(row['purchaseDate']['$date'], unit='ms')
    f_date = pd.to_datetime(row['finishedDate']['$date'], unit='ms')
    if p_date > f_date:
        print('Logical error at index: ' + str(index))

Logical error at index: 14
Logical error at index: 85
Logical error at index: 244
Logical error at index: 362
Logical error at index: 553
Logical error at index: 644
Logical error at index: 871


I would not expect purchaseDate to be able to be after finishedDate, which raises red flags about data quality.

Since we're going to make a new table for Items, we can drop that column from the Receipts table. As purchasedItemCount and totalSpent are only transitively dependent on the primary key and thereform violate normal form, we should drop those columns as well.

In [13]:
receipts = receipts.drop(['rewardsReceiptItemList', 'purchasedItemCount', 'totalSpent'], axis=1)

Lastly, we can clean up the formatting for when we want to query the data in SQL.

In [14]:
temp_field = []
for row in receipts['_id']:
    temp_value = row['$oid']
    temp_field.append(temp_value)
receipts['receiptID'] = temp_field
receipts.insert(0, 'receiptID', receipts.pop('receiptID'))
receipts = receipts.drop(['_id'], axis=1)

In [15]:
for i in ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']:
    temp_field = []
    for row in receipts[i]:
        try:
            temp_value = pd.to_datetime(row['$date'], unit='ms')
            temp_field.append(temp_value)
        except:
            temp_field.append(None)
    receipts[i] = temp_field

In [16]:
receipts = receipts.rename(columns={'userId': 'userID'})

In [17]:
receipts.head()

,receiptID,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,rewardsReceiptStatus,userID
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,FINISHED,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,FINISHED,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,REJECTED,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,FINISHED,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,FINISHED,5ff1e194b6a9d73a3a9f1052


# Items on Receipt Table

To assess the data quality of the items on the receipts, we first create a table with a row for each item. The primary key will be the combination of a receipt ID and a barcode.

In [18]:
def convert(row):
    new_rows = []
    for item in row['rewardsReceiptItemList']:
        item_on_rec = item.copy()
        item_on_rec['receiptID'] = row['_id']['$oid']
        new_rows.append(item_on_rec)
    return new_rows

In [19]:
receipts_occupied = receipts_for_items.dropna(subset=['rewardsReceiptItemList'])

item_receipt_rows = []
for index, row in receipts_occupied.iterrows():
    item_receipt_rows.append(convert(row))

In [20]:
item_receipt_rows = [elem for item in item_receipt_rows for elem in item]

In [21]:
item_on_receipt = pd.DataFrame.from_dict(item_receipt_rows)

In [22]:
item_on_receipt.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now that the table exists, we should make sure that the primary keys are unique. This is already a problem, as nearly half of the values in the barcode column are null. We can try to make an itemID column that borrows barcodes from userFlaggedBarcode or originalMetaBriteBarcode if necessary.

In [23]:
print(len(item_on_receipt))

item_on_receipt.count()

6941


barcode                               3090
description                           6560
finalPrice                            6767
itemPrice                             6767
needsFetchReview                       813
partnerItemId                         6941
preventTargetGapPoints                 358
quantityPurchased                     6767
userFlaggedBarcode                     337
userFlaggedNewItem                     323
userFlaggedPrice                       299
userFlaggedQuantity                    299
receiptID                             6941
needsFetchReviewReason                 219
pointsNotAwardedReason                 340
pointsPayerId                         1267
rewardsGroup                          1731
rewardsProductPartnerId               2269
userFlaggedDescription                 205
originalMetaBriteBarcode                71
originalMetaBriteDescription            10
brandCode                             2600
competitorRewardsGroup                 275
discountedI

In [24]:
# Adding two boolean columns to indicate whether the alternate barcode was used for the itemID column
item_on_receipt['userFlaggedBarcodeBool'] = pd.isna(item_on_receipt['barcode']) & ~pd.isna(item_on_receipt['userFlaggedBarcode'])
item_on_receipt['originalMetaBriteBarcodeBool'] = pd.isna(item_on_receipt['barcode']) & pd.isna(item_on_receipt['userFlaggedBarcode']) & ~pd.isna(item_on_receipt['originalMetaBriteBarcode'])

In [25]:
itemids = []
for index,row in item_on_receipt.iterrows():
    if type(row['barcode']) == str:
        itemids.append(row['barcode'])
    elif row['userFlaggedBarcodeBool']:
        itemids.append(row['userFlaggedBarcode'])
    elif row['originalMetaBriteBarcodeBool']:
        itemids.append(row['originalMetaBriteBarcode'])
    else:
        itemids.append(None)

In [26]:
item_on_receipt['itemID'] = itemids

In [27]:
print(len(item_on_receipt))

item_on_receipt.count()

6941


barcode                               3090
description                           6560
finalPrice                            6767
itemPrice                             6767
needsFetchReview                       813
partnerItemId                         6941
preventTargetGapPoints                 358
quantityPurchased                     6767
userFlaggedBarcode                     337
userFlaggedNewItem                     323
userFlaggedPrice                       299
userFlaggedQuantity                    299
receiptID                             6941
needsFetchReviewReason                 219
pointsNotAwardedReason                 340
pointsPayerId                         1267
rewardsGroup                          1731
rewardsProductPartnerId               2269
userFlaggedDescription                 205
originalMetaBriteBarcode                71
originalMetaBriteDescription            10
brandCode                             2600
competitorRewardsGroup                 275
discountedI

Still only 3,240 values have a valid itemID. Let's drop the null values and move the primary keys to the front to continue testing them.

In [28]:
item_on_receipt = item_on_receipt[~pd.isna(item_on_receipt['itemID'])]

In [29]:
item_on_receipt.insert(0, 'receiptID', item_on_receipt.pop('receiptID'))
item_on_receipt.insert(0, 'itemID', item_on_receipt.pop('itemID'))

In [30]:
print(len(item_on_receipt))

len(set([(row['itemID'],row['receiptID']) for index,row in item_on_receipt.iterrows()]))

3240


1938

I found that there are many repeats and investigated further. Multiple examples of the same item are sometimes counted together and other times counted seperately. To resolve this, we can use partnerItemId as a third aspect of the primary key. After making that change, all primary keys are unique.

In [31]:
s = [(row['itemID'],row['receiptID']) for index,row in item_on_receipt.iterrows()]
Counter(s).most_common()[0]

(('021000658831', '600f2fc80a720f0535000030'), 21)

In [32]:
item_on_receipt[(item_on_receipt['receiptID'] == '600f2fc80a720f0535000030') & (item_on_receipt['itemID'] == '021000658831')].head()

,itemID,receiptID,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,userFlaggedBarcodeBool,originalMetaBriteBarcodeBool
6584,021000658831,600f2fc80a720f0535000030,021000658831,Kraft Macaroni & Cheese Dinner - The Cheesiest...,1.00,1.00,False,1882,NaN,1.0,...,NaN,NaN,True,NaN,NaN,NaN,NaN,KRAFT MACARONI & CHEESE DINNER - BOX,False,False
6585,021000658831,600f2fc80a720f0535000030,021000658831,Kraft Macaroni & Cheese Dinner - The Cheesiest...,1.00,1.00,False,1884,NaN,1.0,...,NaN,NaN,True,NaN,NaN,NaN,NaN,KRAFT MACARONI & CHEESE DINNER - BOX,False,False
6586,021000658831,600f2fc80a720f0535000030,021000658831,Kraft Macaroni & Cheese Dinner - The Cheesiest...,1.00,1.00,False,1888,NaN,1.0,...,NaN,NaN,True,NaN,NaN,NaN,NaN,KRAFT MACARONI & CHEESE DINNER - BOX,False,False
6590,021000658831,600f2fc80a720f0535000030,021000658831,Kraft Macaroni & Cheese Dinner - The Cheesiest...,1.00,1.00,False,1904,NaN,1.0,...,NaN,NaN,True,NaN,NaN,NaN,NaN,KRAFT MACARONI & CHEESE DINNER - BOX,False,False
6591,021000658831,600f2fc80a720f0535000030,021000658831,Kraft Macaroni & Cheese Dinner - The Cheesiest...,1.00,1.00,False,1907,NaN,1.0,...,NaN,NaN,True,NaN,NaN,NaN,NaN,KRAFT MACARONI & CHEESE DINNER - BOX,False,False


In [33]:
print(len(item_on_receipt))

len(set([(row['itemID'],row['receiptID'],row['partnerItemId']) for index,row in item_on_receipt.iterrows()]))

3240


3240

In [34]:
item_on_receipt.insert(2, 'partnerItemId', item_on_receipt.pop('partnerItemId'))

I wondered if the description field is dependent only on barcode/itemID (and would therefore require a new Item-Description table), but it turned out not to be.

In [35]:
len(set(item_on_receipt['itemID']))

568

In [36]:
len(set(item_on_receipt['description']))

556

Now that itemID exists as an aggregate of data from the various barcodes (and we have columns to indicate when a barcode was pulled from a given source), we can drop those columns.

In [37]:
item_on_receipt = item_on_receipt.drop(['barcode', 'userFlaggedBarcode', 'originalMetaBriteBarcode'], axis=1)

In [38]:
item_on_receipt.head()

,itemID,receiptID,partnerItemId,description,finalPrice,itemPrice,needsFetchReview,preventTargetGapPoints,quantityPurchased,userFlaggedNewItem,...,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,userFlaggedBarcodeBool,originalMetaBriteBarcodeBool
0,4011,5ff1e1eb0a720f0523000575,1,ITEM NOT FOUND,26.00,26.00,False,True,5.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,4011,5ff1e1bb0a720f052300056b,1,ITEM NOT FOUND,1,1,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,028400642255,5ff1e1bb0a720f052300056b,2,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,True,1.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,4011,5ff1e1f10a720f052300057a,1,NaN,NaN,NaN,False,True,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4,4011,5ff1e1ee0a7214ada100056f,1,ITEM NOT FOUND,28.00,28.00,False,True,4.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


# User Table

In [39]:
users = pd.read_json('users.json', lines=True)
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


We will again start by checking the primary key.

In [40]:
print(len(users))

list_of_usids = [row['$oid'] for row in users['_id']]
print(len(set(list_of_usids)))

495
212


In [41]:
Counter(list_of_usids).most_common()[0]

('5fc961c3b8cfca11a077dd33', 20)

In [42]:
users[users['_id']=={'$oid': '5fc961c3b8cfca11a077dd33'}][:5]

,_id,active,createdDate,lastLogin,role,signUpSource,state
435,{'$oid': '5fc961c3b8cfca11a077dd33'},True,{'$date': 1607033283936},{'$date': 1614379156799},fetch-staff,Email,NH
436,{'$oid': '5fc961c3b8cfca11a077dd33'},True,{'$date': 1607033283936},{'$date': 1614379156799},fetch-staff,Email,NH
437,{'$oid': '5fc961c3b8cfca11a077dd33'},True,{'$date': 1607033283936},{'$date': 1614379156799},fetch-staff,Email,NH
438,{'$oid': '5fc961c3b8cfca11a077dd33'},True,{'$date': 1607033283936},{'$date': 1614379156799},fetch-staff,Email,NH
439,{'$oid': '5fc961c3b8cfca11a077dd33'},True,{'$date': 1607033283936},{'$date': 1614379156799},fetch-staff,Email,NH


The primary keys are not unique, but this might only apply to those with the fetch-staff role.

In [43]:
nonstaff = users[users['role']!='fetch-staff']

In [44]:
print(len(nonstaff))

list_of_usids = [row['$oid'] for row in nonstaff['_id']]
print(len(set(list_of_usids)))

413
204


In [45]:
Counter(list_of_usids).most_common()[0]

('5ff5d15aeb7c7d12096d91a2', 18)

In [46]:
users[users['_id']=={'$oid': '5ff5d15aeb7c7d12096d91a2'}][:5]

,_id,active,createdDate,lastLogin,role,signUpSource,state
60,{'$oid': '5ff5d15aeb7c7d12096d91a2'},True,{'$date': 1609945434680},{'$date': 1609945690009},consumer,Email,WI
61,{'$oid': '5ff5d15aeb7c7d12096d91a2'},True,{'$date': 1609945434680},{'$date': 1609945690009},consumer,Email,WI
62,{'$oid': '5ff5d15aeb7c7d12096d91a2'},True,{'$date': 1609945434680},{'$date': 1609945690009},consumer,Email,WI
63,{'$oid': '5ff5d15aeb7c7d12096d91a2'},True,{'$date': 1609945434680},{'$date': 1609945690009},consumer,Email,WI
64,{'$oid': '5ff5d15aeb7c7d12096d91a2'},True,{'$date': 1609945434680},{'$date': 1609945690009},consumer,Email,WI


It turns out consumers have repeat appearances too. The cause of this necessitates further investigation, but for now, it may be best to remove duplicate users.

In [47]:
users['id_flat'] = [row['$oid'] for row in users['_id']]

In [48]:
users = users.drop_duplicates(['id_flat', 'state', 'role'])

In [49]:
print(len(users))

list_of_usids = [row['$oid'] for row in users['_id']]
print(len(set(list_of_usids)))

212
212


For the users table, we can also assess data quality by comparing the date fields. These fields are logically consistent.

In [50]:
for index,row in users[~pd.isna(users['lastLogin'])].iterrows():
    c_date = pd.to_datetime(row['createdDate']['$date'], unit='ms')
    f_date = pd.to_datetime(row['lastLogin']['$date'], unit='ms')
    if c_date > f_date:
        print('Logical error at index: ' + str(index))

Lastly, we'll do some formatting of the IDs and dates.

In [51]:
temp_field = []
for row in users['_id']:
    temp_value = row['$oid']
    temp_field.append(temp_value)
users['userID'] = temp_field
users.insert(0, 'userID', users.pop('userID'))
users = users.drop(['id_flat', '_id'], axis=1)

In [52]:
for i in ['createdDate', 'lastLogin']:
    temp_field = []
    for row in users[i]:
        try:
            temp_value = pd.to_datetime(row['$date'], unit='ms')
            temp_field.append(temp_value)
        except:
            temp_field.append(None)
    users[i] = temp_field

In [53]:
users.head()

,userID,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 15:24:39.626,2021-01-03 15:24:39.665,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599,consumer,Email,WI


# Brand Table

In [54]:
brands = pd.read_json('brands.json', lines=True)
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


All primary keys are unique.

In [55]:
print(len(brands))

list_of_brandsids = [row['$oid'] for row in brands['_id']]
print(len(set(list_of_brandsids)))

1167
1167


In [56]:
brands.count()

_id             1167
barcode         1167
category        1012
categoryCode     517
cpg             1167
name            1167
topBrand         555
brandCode        933
dtype: int64

Several fields are missing values, notably categoryCode and brandCode. topBrand could be correct, as some values should be zero (if they are not a top brand).

In addition, the category and categoryCode fields look like they might have a transitive dependence to the primary key. A new table for Category Codes and their respective Category Names should probably be created to address that.

In [57]:
brands.groupby('categoryCode')['category'].unique()

categoryCode
BABY                                                    [Baby]
BAKING                                                [Baking]
BEER_WINE_SPIRITS                          [Beer Wine Spirits]
BEVERAGES                                          [Beverages]
BREAD_AND_BAKERY                              [Bread & Bakery]
CANDY_AND_SWEETS                              [Candy & Sweets]
CLEANING_AND_HOME_IMPROVEMENT    [Cleaning & Home Improvement]
DAIRY_AND_REFRIGERATED                  [Dairy & Refrigerated]
FROZEN                                                [Frozen]
GROCERY                                              [Grocery]
HEALTHY_AND_WELLNESS                       [Health & Wellness]
MAGAZINES                                          [Magazines]
OUTDOOR                                              [Outdoor]
PERSONAL_CARE                                  [Personal Care]
Name: category, dtype: object

In [58]:
# Saving a copy of brands table for creation of category table
brands_copy = brands.copy()

Some barcodes fall under multiple brands, which may be a data issue. I would expect each barcode to be unique to a brand. I would ask Product to clarify the cause of this.

In [59]:
brands.groupby('barcode').count().sort_values('_id', ascending=False)

,_id,category,categoryCode,cpg,name,topBrand,brandCode
barcode,,,,,,,
511111504139,2,2,0,2,2,1,2
511111305125,2,2,0,2,2,0,2
511111704140,2,1,0,2,2,1,2
511111605058,2,2,0,2,2,1,2
511111004790,2,2,0,2,2,1,2
...,...,...,...,...,...,...,...
511111303893,1,1,0,1,1,1,1
511111303732,1,0,0,1,1,0,0
511111303664,1,0,0,1,1,0,0


In the meantime, we can just drop categoryCode as it seems to be the less complete field between itself and category.

In [60]:
brands = brands.drop('categoryCode', axis=1)

In [61]:
brands.head()

,_id,barcode,category,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


Final formatting for SQL:

In [62]:
temp_field = []
for row in brands['_id']:
    temp_value = row['$oid']
    temp_field.append(temp_value)
brands['brandID'] = temp_field
brands.insert(0, 'brandID', brands.pop('brandID'))
brands = brands.drop('_id', axis=1)

In [63]:
temp_field = []
temp_field2 = []
for row in brands['cpg']:
    try:
        temp_value = row['$id']['$oid']
        temp_field.append(temp_value)
        temp_value2 = row['$ref']
        temp_field2.append(temp_value2)
    except:
        temp_field.append(None)
        temp_field2.append(None)
brands['cpg_id'] = temp_field
brands['cpg_ref'] = temp_field2
brands = brands.drop('cpg', axis=1)

In [64]:
brands = brands.rename({'barcode': 'itemID'}, axis=1)

In [65]:
brands['itemID'] = brands['itemID'].apply(str)

In [66]:
brands.head()

,brandID,itemID,category,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


# Categories Table

In [67]:
categories = brands_copy.drop(['_id', 'barcode', 'cpg', 'name', 'topBrand', 'brandCode'], axis=1)
categories = categories.dropna()
categories = categories.drop_duplicates()
print(len(categories))
print(len(categories['category'].unique()))
categories.head()

14
14


,category,categoryCode
0,Baking,BAKING
1,Beverages,BEVERAGES
4,Candy & Sweets,CANDY_AND_SWEETS
16,Health & Wellness,HEALTHY_AND_WELLNESS
68,Grocery,GROCERY


# SQL Querying for Stakeholder Questions

In [68]:
!pip install ipython-sql

In [69]:
cnn = sqlite3.connect('receipts_data.db')

receipts.to_sql('receipts', cnn, if_exists='replace')
item_on_receipt.to_sql('items', cnn, if_exists='replace')
users.to_sql('users', cnn, if_exists='replace')
brands.to_sql('brands', cnn, if_exists='replace')
categories.to_sql('categories', cnn, if_exists='replace')

In [70]:
%load_ext sql

%sql sqlite:///receipts_data.db

# Questions 1-2
What are the top 5 brands by receipts scanned for most recent month?

How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

The most recent month is March 2021 and the second most recent is February 2021. However, due to the data quality issues, January is the only month that populates a list of brands, so I also included that month as proof of the code function.

In [71]:
%%sql

SELECT strftime('%m', purchaseDate) as month, strftime('%Y', purchaseDate) as year, COUNT(DISTINCT receiptID) as receipt_count
FROM receipts
GROUP BY month, year
ORDER BY receipt_count DESC;

 * sqlite:///receipts_data.db
Done.


month,year,receipt_count
01,2021,498
None,None,448
02,2021,87
08,2020,40
12,2020,25
10,2017,9
11,2020,6
03,2021,2
10,2020,2
01,2019,1


There are many more receipts in January 2021 than any other month - even before considering missing data.

In [72]:
%%sql

SELECT brands.name, COUNT(DISTINCT items.receiptID) AS receipt_count
FROM items
JOIN brands ON items.itemID = brands.itemID
JOIN receipts ON items.receiptID = receipts.receiptID
WHERE strftime('%m', receipts.purchaseDate) = '03' AND strftime('%Y', receipts.purchaseDate) = '2021'
GROUP BY items.brandCode
ORDER BY receipt_count DESC
LIMIT 5;

 * sqlite:///receipts_data.db
Done.


name,receipt_count


In [73]:
%%sql

SELECT brands.name, COUNT(DISTINCT items.receiptID) AS receipt_count
FROM items
JOIN brands ON items.itemID = brands.itemID
JOIN receipts ON items.receiptID = receipts.receiptID
WHERE strftime('%m', receipts.purchaseDate) = '02' AND strftime('%Y', receipts.purchaseDate) = '2021'
GROUP BY items.brandCode
ORDER BY receipt_count DESC
LIMIT 5;

 * sqlite:///receipts_data.db
Done.


name,receipt_count


In [74]:
%%sql

SELECT brands.name, COUNT(DISTINCT items.receiptID) AS receipt_count
FROM items
JOIN brands ON items.itemID = brands.itemID
JOIN receipts ON items.receiptID = receipts.receiptID
WHERE strftime('%m', receipts.purchaseDate) = '01' AND strftime('%Y', receipts.purchaseDate) = '2021'
GROUP BY items.brandCode
ORDER BY receipt_count DESC
LIMIT 5;

 * sqlite:///receipts_data.db
Done.


name,receipt_count
Tostitos,11
Swanson,11
Cracker Barrel Cheese,10
Diet Chris Cola,4
Kraft,3


# Questions 3-4
When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [75]:
%%sql

SELECT rewardsReceiptStatus, AVG(total_spend) as avg_total_spend
FROM (
SELECT receipts.receiptID, receipts.rewardsReceiptStatus, SUM(items.finalPrice) as total_spend
FROM receipts
JOIN items ON receipts.receiptID = items.receiptID
GROUP BY receipts.receiptID
)
GROUP BY rewardsReceiptStatus;

 * sqlite:///receipts_data.db
Done.


rewardsReceiptStatus,avg_total_spend
FINISHED,41.54855421686751
FLAGGED,102.7654285714286
REJECTED,26.204500000000014


In [76]:
%%sql

SELECT rewardsReceiptStatus, AVG(item_count) as avg_item_quantity
FROM (
SELECT receipts.receiptID, receipts.rewardsReceiptStatus, COUNT(items.itemID) as item_count
FROM receipts
JOIN items ON receipts.receiptID = items.receiptID
GROUP BY receipts.receiptID
)
GROUP BY rewardsReceiptStatus;

 * sqlite:///receipts_data.db
Done.


rewardsReceiptStatus,avg_item_quantity
FINISHED,5.520080321285141
FLAGGED,7.688888888888889
REJECTED,2.265625


"Finished" receipts had 59% higher spend and 144% more items than "Rejected" receipts.

# Questions 5-6
Which brand has the most spend among users who were created within the past 6 months?

Which brand has the most transactions among users who were created within the past 6 months?

In [77]:
%%sql

SELECT name, AVG(total_spend) as avg_total_spend
FROM (
SELECT brands.name, SUM(items.finalPrice) as total_spend
FROM brands
JOIN items ON brands.brandCode = items.brandCode
JOIN receipts ON items.receiptID = receipts.receiptID
JOIN users on receipts.userID = users.userID
WHERE julianday('now') - julianday(users.createdDate) <= 180
GROUP BY brands.name
)
GROUP BY name
ORDER BY avg_total_spend DESC
LIMIT 1;

 * sqlite:///receipts_data.db
Done.


name,avg_total_spend


In [78]:
%%sql

SELECT name, AVG(total_spend) as avg_total_spend
FROM (
SELECT brands.name, SUM(items.finalPrice) as total_spend
FROM brands
JOIN items ON brands.brandCode = items.brandCode
JOIN receipts ON items.receiptID = receipts.receiptID
JOIN users on receipts.userID = users.userID
WHERE julianday('now') - julianday(users.createdDate) <= 365*10
GROUP BY brands.name
)
GROUP BY name
ORDER BY avg_total_spend DESC
LIMIT 1;

 * sqlite:///receipts_data.db
Done.


name,avg_total_spend
KNORR,548.1800000000001


In [79]:
%%sql

SELECT brands.name, COUNT(DISTINCT receipts.receiptID) as transactions
FROM brands
JOIN items ON brands.brandCode = items.brandCode
JOIN receipts ON items.receiptID = receipts.receiptID
JOIN users on receipts.userID = users.userID
WHERE julianday('now') - julianday(users.createdDate) <= 180
GROUP BY brands.name
ORDER BY transactions DESC
LIMIT 1;

 * sqlite:///receipts_data.db
Done.


name,transactions


In [80]:
%%sql

SELECT brands.name, COUNT(DISTINCT receipts.receiptID) as transactions
FROM brands
JOIN items ON brands.brandCode = items.brandCode
JOIN receipts ON items.receiptID = receipts.receiptID
JOIN users on receipts.userID = users.userID
WHERE julianday('now') - julianday(users.createdDate) <= 365*10
GROUP BY brands.name
ORDER BY transactions DESC
LIMIT 1;

 * sqlite:///receipts_data.db
Done.


name,transactions
Kleenex,17
